# Hypothesis Testing

In this notebook we demonstrate formal hypothesis testing using the NHANES data.

It is important to note that the NHANES data are a "complex survey".  The data are not an independent and representative sample from the target population.  Proper analysis of complex survey data should make use of additional information about how the data were collected.  Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg') # workaround, there may be a better way
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Below we read the data, and convert some of the integer codes to text values.

In [2]:
url = "https://raw.githubusercontent.com/lenos16/Inferrential-Statistics-with-Python/main/nhanes_2015_2016.csv"
da = pd.read_csv(url)

da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})

In [3]:
da["SMQ020x"].head()

0    Yes
1    Yes
2    Yes
3     No
4     No
Name: SMQ020x, dtype: object

In [4]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

da["RIAGENDRx"].head()

0      Male
1      Male
2      Male
3    Female
4    Female
Name: RIAGENDRx, dtype: object

### Hypothesis Tests for One Proportion

The most basic hypothesis test may be the one-sample test for a proportion.  This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value.  One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison. For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%.  In the following notebook cell, we carry out the (two-sided) one-sample test that the population proportion of smokers is 0.4, and obtain a p-value of 0.43.  This indicates that the NHANES data are compatible with the proportion of (ever) smokers in the US being 40%. 

In [5]:
x=da.SMQ020x.dropna()=="Yes"

In [6]:
p=x.mean()
p

0.4050655021834061

In [8]:
se=np.sqrt(.4*(1-.4)/len(x))
se

0.00647467353462031

In [10]:
test_stat=(p-.4)/se
test_stat

0.7823563854332805

In [11]:
p_value=2*dist.norm.cdf(-np.abs(test_stat))
print(test_stat,p_value)

0.7823563854332805 0.4340051581348052


In [ ]:
x = da.SMQ020x.dropna() == "Yes"

In [ ]:
p = x.mean()

In [ ]:
p

In [ ]:
se = np.sqrt(.4 * (1 - .4)/ len(x))
se

In [ ]:
test_stat = (p - 0.4) / se
test_stat

In [ ]:
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pvalue)

The following cell carries out the same test as performed above using the Statsmodels library.  The results in the first (default) case below are slightly different from the results obtained above because Statsmodels by default uses the sample proportion instead of the null proportion when computing the standard error.  This distinction is rarely consequential, but we can specify that the null proportion should be used to calculate the standard error, and the results agree exactly with what we calculated above.  The first two lines below carry out tests using the normal approximation to the sampling distribution of the test statistic, and the third line below carries uses the exact binomial sampling distribution.  We can see here that the p-values are nearly identical in all three cases. This is expected when the sample size is large, and the proportion is not close to either 0 or 1.

In [12]:
print(sm.stats.proportions_ztest(x.sum(),len(x),0.4)) #Normal approximation with estimated proportions
print(sm.stats.binom_test(x.sum(),len(x),0.4)) #Excat binomial p-value

(0.7807518954896244, 0.43494843171868214)
0.4340360854459431


In [ ]:
sm.stats.proportions_ztest(x.sum(), len(x), 0.4)

In [ ]:
sm.stats.binom_test(x.sum(), len(x), 0.4)

### Hypothesis Tests for Two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value.  A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations.  For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age.  In the cell below, we carry out this test without using any libraries, implementing all the test procedures covered elsewhere in the course using Python code.  We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [14]:
dx=da[["SMQ020x","RIDAGEYR","RIAGENDRx"]].dropna()
dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [15]:
dx = da[["SMQ020x", "RIDAGEYR", "RIAGENDRx"]].dropna()

dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [16]:
#Sumamrize the data by calculating the proportion of yes responses and the sample size
p=dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z=="Yes"),"size"])
p.columns=["Smoke","N"]
print(p)

              Smoke     N
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753


In [ ]:
p = dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z == "Yes"), "size"])
p.columns = ["Smoke", "N"]
print(p)

Essentially the same test as above can be conducted by converting the "Yes"/"No" responses to numbers (Yes=1, No=0) and conducting a two-sample t-test, as below:

In [20]:
# The pooled rate of yes responses, and the standard error of the estimated difference of proportions between male and female
p_comb=(dx.SMQ020x=="Yes").mean()
va=p_comb*(1-p_comb)
se=np.sqrt(va*(1/p.N.Female+1/p.N.Male))
(p_comb, va, se)

(0.4050655021834061, 0.2409874411243111, 0.01298546309757376)

In [21]:
test_stat=(p.Smoke.Female-p.Smoke.Male)/se
p_value=2*dist.norm.cdf(-np.abs(test_stat))
(test_stat, p_value)

(-16.049719603652488, 5.742288777302776e-58)

In [ ]:
p_comb = (dx.SMQ020x == "Yes").mean()
va = p_comb * (1 - p_comb)

se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))

In [ ]:
(p_comb, va, se)

In [19]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
(test_stat, p_value)

(-16.049719603652488, 5.742288777302776e-58)

In [24]:
dx_females=dx.loc[dx.RIAGENDRx=="Female","SMQ020x"].replace({"Yes":1,"No":0})
dx_males=dx.loc[dx.RIAGENDRx=="Male","SMQ020x"].replace({"Yes":1,"No":0})
dx_females
dx_males

0       1
1       1
2       1
6       1
8       0
       ..
5726    1
5728    0
5729    0
5731    0
5733    1
Name: SMQ020x, Length: 2753, dtype: int64

In [25]:
sm.stats.ttest_ind(dx_females,dx_males) #prints test statistic, p-value, degree of freedom

(-16.420585558984445, 3.0320887866906843e-59, 5723.0)

In [ ]:
dx_females = dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_females

In [ ]:
dx_males = dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_males

In [ ]:
sm.stats.ttest_ind(dx_females, dx_males)

### Hypothesis Tests Comparing Means

Tests of means are similar in many ways to tests of proportions.  Just as with proportions, for comparing means there are one and two-sample tests, z-tests and t-tests, and one-sided and two-sided tests.  As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below.  We compare systolic blood pressure to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

In [29]:
dx=da[["BPXSY1","RIDAGEYR","RIAGENDRx"]].dropna()
dx
dx=dx.loc[(dx.RIDAGEYR>=40)&(dx.RIDAGEYR<=50)&(dx.RIAGENDRx=="Male"), :]
dx

,BPXSY1,RIDAGEYR,RIAGENDRx
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male
...,...,...,...
5680,134.0,50,Male
5690,138.0,48,Male
5693,96.0,41,Male
5713,116.0,43,Male


In [30]:
print(dx.BPXSY1.mean())

125.86698337292161


In [31]:
sm.stats.ztest(dx.BPXSY1, value=120)

(7.469764137102597, 8.033869113167905e-14)

In [ ]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx

In [ ]:
dx = dx.loc[(dx.RIDAGEYR >= 40) & (dx.RIDAGEYR <= 50) & (dx.RIAGENDRx == "Male"), :]
dx

In [ ]:
print(dx.BPXSY1.mean())

In [ ]:
sm.stats.ztest(dx.BPXSY1, value=120)

In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60.  The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant. 

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test.  Below we display results from both these testing approaches.  When the sample size is large, the difference between the t-test and z-test is very small.  

In [32]:
dx=da[["BPXSY1","RIDAGEYR","RIAGENDRx"]].dropna()
dx=dx.loc[(dx.RIDAGEYR>=50) & (dx.RIDAGEYR<=60),:]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [33]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [35]:
bpx_female=dx.loc[(dx.RIAGENDRx=="Female","BPXSY1")]
bpx_male=dx.loc[(dx.RIAGENDRx=="Male","BPXSY1")]
print(bpx_female.mean(), bpx_male.mean()) #female mean, male mean

127.92561983471074 129.23829787234044


In [37]:
print(sm.stats.ztest(bpx_female, bpx_male)) # test statistic, p-value

(-1.105435895556249, 0.2689707570859362)


In [38]:
print(sm.stats.ttest_ind(bpx_female, bpx_male)) # test statistic, p-value, degree of freedom

(-1.105435895556249, 0.26925004137768577, 952.0)


In [ ]:
bpx_female = dx.loc[dx.RIAGENDRx=="Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDRx=="Male", "BPXSY1"]
print(bpx_female.mean(), bpx_male.mean())

In [ ]:
print(sm.stats.ztest(bpx_female, bpx_male))

In [ ]:
print(sm.stats.ttest_ind(bpx_female, bpx_male))

Another important aspect of two-sample mean testing is "heteroscedasticity", meaning that the variances within the two groups being compared may be different. While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statisitically significant). In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands. In every age band, females having greater variation than males. 

In [42]:
dx=da[["BMXBMI","RIDAGEYR","RIAGENDRx"]].dropna()
da["agegrp"]=pd.cut(da.RIDAGEYR,[18,30,40,50,60,70,80])
da.groupby(["agegrp","RIAGENDRx"])["BMXBMI"].agg(np.std).unstack

<bound method Series.unstack of agegrp    RIAGENDRx
(18, 30]  Female       7.745893
          Male         6.649440
(30, 40]  Female       8.315608
          Male         6.622412
(40, 50]  Female       8.076195
          Male         6.407076
(50, 60]  Female       7.575848
          Male         5.914373
(60, 70]  Female       7.604514
          Male         5.933307
(70, 80]  Female       6.284968
          Male         4.974855
Name: BMXBMI, dtype: float64>

The Std between the sexes in each agegroup  is very different. This is proof that heteroscedacity deffinitely exists between these two groups

In [ ]:
dx = da[["BMXBMI", "RIDAGEYR", "RIAGENDRx"]].dropna()
da["agegrp"] = pd.cut(da.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
da.groupby(["agegrp", "RIAGENDRx"])["BMXBMI"].agg(np.std).unstack()

The standard error of the mean difference (e.g. mean female blood pressure minus mean mal blood pressure) can be estimated in at least two different ways. In the statsmodels library, these approaches are referred to as the "pooled" and the "unequal" approach to estimating the variance. If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches. Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two differences are quite similar. Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men. The results printed in each row of output are the test-statistic and p-value. 

In [ ]:
for k, v in da.groupby("agegrp"):
    bmi_female = v.loc[v.RIAGENDRx=="Female", "BMXBMI"].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDRx=="Male", "BMXBMI"].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='pooled'))
    print("unequal: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='unequal'))
    print()